# DES Y6 Deep Field Exposures:  Photometric Zeropoints tied to FGCM

## 1. Setup

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate
import glob
import math
import os
import subprocess
import sys
import gc

import glob
import pickle

import easyaccess as ea

#import AlasBabylon

import fitsio
from astropy.io import fits
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, vstack

import tempfile

import matplotlib.pyplot as plt
 
%matplotlib inline

In [2]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## 2. User Input

### 2.1.  General User Input  

In [3]:
verbose = 1
tag_des = 'Y6A2_FINALCUT'      # Official tag for DES Y6A2_FINALCUT
tag_decade = 'DECADE_FINALCUT' # Tag for DECADE
rawdata_dir = '../RawData'
zeropoints_dir='../Zeropoints'

bandList = ['g', 'r', 'i', 'z', 'Y']

### 2.2. Logical Variables to Indicate which Code Cells to Run

In [4]:
do_calc_fgcm_psf_zps = True
do_calc_fgcm_aper8_zps = True

### 2.3. Sky Region Definitions

In [5]:
region_name_list = [
                    'VVDSF14', 
                    'VVDSF22', 
                    'DEEP2', 
                    'SN-E', 
                    'SN-X_err', 
                    'SN-X', 
                    'ALHAMBRA2', 
                    'SN-S',
                    'SN-C', 
                    'EDFS',
                    'MACS0416',
                    'SN-S_err',
                    'COSMOS'
                    ]

region_ramin = {
                'VVDSF14':208.,
                'VVDSF22':333.,
                'DEEP2':351., 
                'SN-E':6., 
                'SN-X_err':13., 
                'SN-X':32., 
                'ALHAMBRA2':35., 
                'SN-S':39.5,
                'SN-C':50., 
                'EDFS':55.,
                'MACS0416':62.,
                'SN-S_err':83.5,
                'COSMOS':148.
               }

region_ramax = {
                'VVDSF14':212.,
                'VVDSF22':337.,
                'DEEP2':354., 
                'SN-E':12., 
                'SN-X_err':17., 
                'SN-X':38., 
                'ALHAMBRA2':39., 
                'SN-S':44.5,
                'SN-C':56., 
                'EDFS':67.,
                'MACS0416':66.,
                'SN-S_err':88.,
                'COSMOS':153.
               }

region_decmin = {
                'VVDSF14':3.,
                'VVDSF22':-1.5,
                'DEEP2':-1.5, 
                'SN-E':-46., 
                'SN-X_err':-32., 
                'SN-X':-8., 
                'ALHAMBRA2':-0.5, 
                'SN-S':-2.5,
                'SN-C':-31., 
                'EDFS':-51.,
                'MACS0416':-25.5,
                'SN-S_err':-38.,
                'COSMOS':0.5
               }


region_decmax = {
                'VVDSF14':7.,
                'VVDSF22':1.5,
                'DEEP2':1.5, 
                'SN-E':-41., 
                'SN-X_err':-29., 
                'SN-X':-3., 
                'ALHAMBRA2':2.5, 
                'SN-S':1.5,
                'SN-C':-25., 
                'EDFS':-46.,
                'MACS0416':-22.5,
                'SN-S_err':-35.,
                'COSMOS':4.0
               }


for regionName in region_name_list:
    print regionName, region_ramin[regionName], region_ramax[regionName], region_decmin[regionName], region_decmax[regionName]

VVDSF14 208.0 212.0 3.0 7.0
VVDSF22 333.0 337.0 -1.5 1.5
DEEP2 351.0 354.0 -1.5 1.5
SN-E 6.0 12.0 -46.0 -41.0
SN-X_err 13.0 17.0 -32.0 -29.0
SN-X 32.0 38.0 -8.0 -3.0
ALHAMBRA2 35.0 39.0 -0.5 2.5
SN-S 39.5 44.5 -2.5 1.5
SN-C 50.0 56.0 -31.0 -25.0
EDFS 55.0 67.0 -51.0 -46.0
MACS0416 62.0 66.0 -25.5 -22.5
SN-S_err 83.5 88.0 -38.0 -35.0
COSMOS 148.0 153.0 0.5 4.0


### 2.3. Check on Location of TMPDIR...

In [6]:
# Check on TMPDIR... 
tempfile.gettempdir()

'/data/des40.a/data/dtucker/TmpDir'

In [7]:
# Set tmpdir variable to $TMPDIR (for future reference)...
tmpdir = os.environ['TMPDIR']

### 2.4. Create Main Zeropoints Directory (if it does not already exist)...

In [8]:
# Create main Zeropoints directory, if it does not already exist...
if not os.path.exists(zeropoints_dir):
    os.makedirs(zeropoints_dir)

## 3. Useful Modules

In [9]:
def DECam_tie_to_fgcm_stds(inputFile, outputFile, band, magType, fluxObsColName, fluxerrObsColName, aggFieldColName, verbose):

    import numpy as np 
    import os
    import sys
    import datetime
    import pandas as pd
    from astropy.table import Table, vstack
    
    validBandsList = ['g', 'r', 'i', 'z', 'Y']

    if band not in validBandsList:
        print """Filter band %s is not currently handled...  Exiting now!""" % (band)
        return 1
    
    reqColList = ['MAG_STD_G','MAGERR_STD_G',
                  'MAG_STD_R','MAGERR_STD_R',
                  'MAG_STD_I','MAGERR_STD_I',
                  'MAG_STD_Z','MAGERR_STD_Z',
                  'MAG_STD_Y','MAGERR_STD_Y',
                  fluxObsColName,fluxerrObsColName,
                  aggFieldColName]
        
      
    # Does the input file exist?
    if os.path.isfile(inputFile)==False:
        print """DECam_tie_to_fgcm_stds input file %s does not exist.  Exiting...""" % (inputFile)
        return 1

    # Read inputFile into a pandas DataFrame...
    print datetime.datetime.now()
    print """Reading in %s as a pandas DataFrame...""" % (inputFile)
    t = Table.read(inputFile)
    dataFrame = t.to_pandas()
    print datetime.datetime.now()
    print

    reqColFlag = 0
    colList = dataFrame.columns.tolist()
    for reqCol in reqColList:
        if reqCol not in colList:
            print """ERROR:  Required column %s is not in the header""" % (reqCol)
            reqColFlag = 1
    if reqColFlag == 1:
        print """Missing required columns in header of %s...  Exiting now!""" (inputFile)
        return 1

    # Identify column of the standard magnitude for the given band...
    magStdColName = """MAG_STD_%s""" % (band.upper())
    magerrStdColName = """MAGERR_STD_%s""" % (band.upper())


    # Add a 'MAG_OBS' column and a 'MAG_DIFF' column to the pandas DataFrame...
    dataFrame['MAG_OBS'] = -2.5*np.log10(dataFrame[fluxObsColName])
    dataFrame['MAG_DIFF'] = dataFrame[magStdColName]-dataFrame['MAG_OBS']


    ###############################################
    # Aggregate by aggFieldColName
    ###############################################

    # Make a copy of original dataFrame...
    df = dataFrame.copy()

    # Create an initial mask...
    mask1 = ( (df[magStdColName] >= 0.) & (df[magStdColName] <= 25.) )
    mask1 = ( mask1 & (df[fluxObsColName] > 10.) & (df['FLAGS'] < 2) & (np.abs(df['SPREAD_MODEL']) < 0.01))
    if magerrStdColName != 'None':  
        mask1 = ( mask1 & (df[magerrStdColName] < 0.1) )
    magDiffGlobalMedian = df[mask1]['MAG_DIFF'].median()
    magDiffMin = magDiffGlobalMedian - 5.0
    magDiffMax = magDiffGlobalMedian + 5.0
    mask2 = ( (df['MAG_DIFF'] > magDiffMin) & (df['MAG_DIFF'] < magDiffMax) )
    mask = mask1 & mask2

    # Iterate over the copy of dataFrame 3 times, removing outliers...
    #  We are using "Method 2/Group by item" from
    #  http://nbviewer.jupyter.org/urls/bitbucket.org/hrojas/learn-pandas/raw/master/lessons/07%20-%20Lesson.ipynb
    print "Sigma-clipping..."
    niter = 0
    for i in range(3):

        niter = i + 1
        print """   iter%d...""" % ( niter )

        # make a copy of original df, and then delete the old one...
        newdf = df[mask].copy()
        del df
        # group by aggFieldColName...
        grpnewdf = newdf.groupby([aggFieldColName])
        
        # add/update new columns to newdf
        print datetime.datetime.now()
        newdf['Outlier']  = grpnewdf['MAG_DIFF'].transform( lambda x: abs(x-x.mean()) > 3.00*x.std() )
        #newdf['Outlier']  = grpnewdf['MAG_DIFF'].transform( lambda x: abs(x-x.mean()) > 2.00*x.std() )
        print datetime.datetime.now()
        del grpnewdf
        print datetime.datetime.now()
        #print newdf

        nrows = newdf['MAG_DIFF'].size
        print """  Number of rows remaining:  %d""" % ( nrows )

        df = newdf
        mask = ( df['Outlier'] == False )  


    # Perform pandas grouping/aggregating functions on sigma-clipped Data Frame...
    print datetime.datetime.now()
    print 'Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...'
    groupedDataFrame = df.groupby([aggFieldColName])
    magZeroMedian = groupedDataFrame['MAG_DIFF'].median()
    magZeroMean = groupedDataFrame['MAG_DIFF'].mean()
    magZeroStd = groupedDataFrame['MAG_DIFF'].std()
    magZeroNum = groupedDataFrame['MAG_DIFF'].count()
    magZeroErr = magZeroStd/np.sqrt(magZeroNum-1)
    print datetime.datetime.now()
    print

    # Rename these pandas series...
    magZeroMedian.name = 'MAG_ZERO_MEDIAN'
    magZeroMean.name = 'MAG_ZERO_MEAN'
    magZeroStd.name = 'MAG_ZERO_STD'
    magZeroNum.name = 'MAG_ZERO_NUM'
    magZeroErr.name = 'MAG_ZERO_MEAN_ERR'

    # Also, calculate group medians for all columns in df that have a numerical data type...
    numericalColList = df.select_dtypes(include=[np.number]).columns.tolist()
    groupedDataMedian = {}
    for numericalCol in numericalColList:
        groupedDataMedian[numericalCol] = groupedDataFrame[numericalCol].median()
        groupedDataMedian[numericalCol].name = """%s_MEDIAN""" % (numericalCol)

    # Create new data frame containing all the relevant aggregate quantities
    #newDataFrame = pd.concat( [magZeroMedian, magZeroMean, magZeroStd, \
    #                           magZeroErr, magZeroNum], \
    #                           join='outer', axis=1 )
    seriesList = []
    for numericalCol in numericalColList:
        seriesList.append(groupedDataMedian[numericalCol])
    seriesList.extend([magZeroMedian, magZeroMean, magZeroStd, \
                               magZeroErr, magZeroNum])
    #print seriesList
    newDataFrame = pd.concat( seriesList, join='outer', axis=1 )
                               

    #newDataFrame.index.rename('FILENAME', inplace=True)

    # Saving catname-based results to output files...
    print datetime.datetime.now()
    print """Writing %s output file (using pandas to_csv method)...""" % (outputFile)
    newDataFrame.to_csv(outputFile, float_format='%.4f')
    print datetime.datetime.now()
    print

    return 0

## 4. Zeropoints by tying to FGCM PSF Standard Stars

**We will first work with the DES data, and then we will repeat for the DECADE data.**

**DES:**

Calculate zeropoints region by region...

In [10]:
%%time 

if do_calc_fgcm_psf_zps: 
    
    fluxObsColName =    'FLUX_PSF' 
    fluxerrObsColName = 'FLUXERR_PSF' 
    aggFieldColName =   'FILENAME' 
    magType =           'psf'
    
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']


    for regionName in region_name_list:
    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on region %s""" % (regionName)
        print """# # # # # # # # # # # # # # #"""
        print
        
        for band in bandList:
                    
            input_file_template = """cat_%s.%s.?.%s.fgcm_%s.fits""" % (subdir, regionName, band, magType)
            input_file_template = os.path.join(rawdata_dir, 'ExpCatFITS', subdir, input_file_template)
            input_file_list = glob.glob(input_file_template)
            input_file_list = np.sort(input_file_list)

            if np.size(input_file_list) == 0:
                print "No files matching template %s" % (input_file_template)
    
            for inputFile in input_file_list:
            
                print inputFile 

                if os.path.exists(inputFile):

                    #outputFile = os.path.splitext(inputFile)[0] + '.zps.csv'
                    #print outputFile
                    
                    outputFile = os.path.splitext(os.path.basename(inputFile))[0]+'.csv'
                    outputFile = 'zps_' + outputFile[4:]
                    outputFile = os.path.join(zeropoints_dir, outputFile)

                    print outputFile    
                    
                    status = DECam_tie_to_fgcm_stds(inputFile, outputFile, 
                                                    band, magType, 
                                                    fluxObsColName, fluxerrObsColName, 
                                                    aggFieldColName, verbose)
                    if status > 0:
                        print 'ERROR:  %s FAILED!  Continuing...'
            
                else:
            
                    print """%s does not exist...  skipping...""" % (inputFile)
                
                print


# # # # # # # # # # # # # # #
Working on region VVDSF14
# # # # # # # # # # # # # # #

../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.VVDSF14.0.g.fgcm_psf.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.VVDSF14.0.g.fgcm_psf.csv
2021-12-30 20:00:58.030003
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.VVDSF14.0.g.fgcm_psf.fits as a pandas DataFrame...
2021-12-30 20:00:58.386636

Sigma-clipping...
   iter1...
2021-12-30 20:00:58.399396
2021-12-30 20:00:58.402296
2021-12-30 20:00:58.402561
  Number of rows remaining:  0
   iter2...
2021-12-30 20:00:58.406053
2021-12-30 20:00:58.407733
2021-12-30 20:00:58.407856
  Number of rows remaining:  0
   iter3...
2021-12-30 20:00:58.410389
2021-12-30 20:00:58.412052
2021-12-30 20:00:58.412273
  Number of rows remaining:  0
2021-12-30 20:00:58.413086
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-30 20:00:58.416980

2021-12-30 20:00:58.477248
Writing ../Zeropoints/zps_DES_Y6A

2021-12-30 20:01:32.474366

Sigma-clipping...
   iter1...
2021-12-30 20:01:38.440872
2021-12-30 20:01:46.795782
2021-12-30 20:01:46.799074
  Number of rows remaining:  835790
   iter2...
2021-12-30 20:01:50.994447
2021-12-30 20:01:59.386901
2021-12-30 20:01:59.391427
  Number of rows remaining:  816858
   iter3...
2021-12-30 20:02:01.777631
2021-12-30 20:02:09.861537
2021-12-30 20:02:09.864472
  Number of rows remaining:  798896
2021-12-30 20:02:09.866689
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-30 20:02:10.096429

2021-12-30 20:02:16.342311
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-E.0.g.fgcm_psf.csv output file (using pandas to_csv method)...
2021-12-30 20:02:18.691505


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-E.0.r.fgcm_psf.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-E.0.r.fgcm_psf.csv
2021-12-30 20:02:19.172787
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-E.0.r.fgcm_psf.fits a

2021-12-30 20:18:26.063975
2021-12-30 20:18:47.237922
2021-12-30 20:18:47.273358
  Number of rows remaining:  1734259
2021-12-30 20:18:47.300238
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-30 20:18:48.179457

2021-12-30 20:19:00.545359
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-X.0.i.fgcm_psf.csv output file (using pandas to_csv method)...
2021-12-30 20:19:06.652246


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-X.1.i.fgcm_psf.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-X.1.i.fgcm_psf.csv
2021-12-30 20:19:07.395275
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-X.1.i.fgcm_psf.fits as a pandas DataFrame...
2021-12-30 20:20:13.906159

Sigma-clipping...
   iter1...
2021-12-30 20:20:27.540516
2021-12-30 20:20:48.841729
2021-12-30 20:20:48.853829
  Number of rows remaining:  1798511
   iter2...
2021-12-30 20:20:58.151979
2021-12-30 20:21:19.340598
2021-12-30 20:21:19.364738
  Number of rows remai

2021-12-30 20:35:34.640825
2021-12-30 20:35:35.807267
2021-12-30 20:35:35.809307
  Number of rows remaining:  104466
   iter2...
2021-12-30 20:35:35.985398
2021-12-30 20:35:37.176706
2021-12-30 20:35:37.177686
  Number of rows remaining:  101777
   iter3...
2021-12-30 20:35:37.301061
2021-12-30 20:35:38.474303
2021-12-30 20:35:38.475063
  Number of rows remaining:  99769
2021-12-30 20:35:38.476187
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-30 20:35:38.509530

2021-12-30 20:35:38.928024
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.ALHAMBRA2.0.i.fgcm_psf.csv output file (using pandas to_csv method)...
2021-12-30 20:35:39.257184


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.ALHAMBRA2.0.z.fgcm_psf.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.ALHAMBRA2.0.z.fgcm_psf.csv
2021-12-30 20:35:39.282444
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.ALHAMBRA2.0.z.fgcm_psf.fits as a pandas DataFrame...
2021-12-30 20:3

2021-12-30 20:50:41.553394


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-C.0.i.fgcm_psf.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-C.0.i.fgcm_psf.csv
2021-12-30 20:50:42.881305
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-C.0.i.fgcm_psf.fits as a pandas DataFrame...
2021-12-30 20:51:55.308436

Sigma-clipping...
   iter1...
2021-12-30 20:52:10.895817
2021-12-30 20:52:40.095532
2021-12-30 20:52:40.122828
  Number of rows remaining:  2170907
   iter2...
2021-12-30 20:52:47.615999
2021-12-30 20:53:12.349334
2021-12-30 20:53:12.363025
  Number of rows remaining:  2113360
   iter3...
2021-12-30 20:53:19.298320
2021-12-30 20:53:43.934799
2021-12-30 20:53:43.949216
  Number of rows remaining:  2051352
2021-12-30 20:53:43.959965
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-30 20:53:44.738776

2021-12-30 20:53:58.748321
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-C.0.i.fgcm_psf.csv output file (usin

2021-12-30 21:13:20.768075

Sigma-clipping...
   iter1...
2021-12-30 21:13:21.335705
2021-12-30 21:13:22.823427
2021-12-30 21:13:22.824221
  Number of rows remaining:  147687
   iter2...
2021-12-30 21:13:23.168507
2021-12-30 21:13:24.505512
2021-12-30 21:13:24.506293
  Number of rows remaining:  144305
   iter3...
2021-12-30 21:13:24.799803
2021-12-30 21:13:26.144657
2021-12-30 21:13:26.145438
  Number of rows remaining:  141618
2021-12-30 21:13:26.146749
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-30 21:13:26.190300

2021-12-30 21:13:26.801760
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.MACS0416.0.g.fgcm_psf.csv output file (using pandas to_csv method)...
2021-12-30 21:13:27.184856


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.MACS0416.0.r.fgcm_psf.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.MACS0416.0.r.fgcm_psf.csv
2021-12-30 21:13:27.217932
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.MACS0416.0.r

2021-12-30 21:15:26.810351
2021-12-30 21:15:26.812082
  Number of rows remaining:  408396
   iter3...
2021-12-30 21:15:27.516994
2021-12-30 21:15:31.452299
2021-12-30 21:15:31.453927
  Number of rows remaining:  401219
2021-12-30 21:15:31.457282
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-30 21:15:31.582497

2021-12-30 21:15:33.440915
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.COSMOS.0.i.fgcm_psf.csv output file (using pandas to_csv method)...
2021-12-30 21:15:34.556815


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.COSMOS.0.z.fgcm_psf.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.COSMOS.0.z.fgcm_psf.csv
2021-12-30 21:15:34.688291
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.COSMOS.0.z.fgcm_psf.fits as a pandas DataFrame...
2021-12-30 21:16:06.972261

Sigma-clipping...
   iter1...
2021-12-30 21:16:10.819328
2021-12-30 21:16:19.683179
2021-12-30 21:16:19.691307
  Number of rows remaining:  910095
   iter

Combine region-by-region results into a single file...

In [15]:
%%time 

if do_calc_fgcm_psf_zps: 
    
    magType = 'psf'
    
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']

    
    for band in bandList:
                    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
        print

        outputFile = """zps_%s.%s.fgcm_%s.csv""" % (subdir, band, magType)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
    
        input_file_template = """zps_%s.*.?.%s.fgcm_%s.csv""" % (subdir, band, magType)
        input_file_template = os.path.join(zeropoints_dir, input_file_template)
        input_file_list = glob.glob(input_file_template)
        input_file_list = np.sort(input_file_list)

        if np.size(input_file_list) == 0:
            print "No files matching template %s" % (input_file_template)
            continue
    
        df_comb = pd.concat(pd.read_csv(inputFile) for inputFile in input_file_list)

        outputFile = """zps_%s.%s.fgcm_%s.csv""" % (subdir, band, magType)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
        print outputFile
        df_comb.to_csv(outputFile, index=False)
        
        del df_comb



# # # # # # # # # # # # # # #
Working on band g
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.g.fgcm_psf.csv

# # # # # # # # # # # # # # #
Working on band r
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.r.fgcm_psf.csv

# # # # # # # # # # # # # # #
Working on band i
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.i.fgcm_psf.csv

# # # # # # # # # # # # # # #
Working on band z
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.z.fgcm_psf.csv

# # # # # # # # # # # # # # #
Working on band Y
# # # # # # # # # # # # # # #

No files matching template ../Zeropoints/zps_DES_Y6A2_FINALCUT.*.?.Y.fgcm_psf.csv
CPU times: user 1min 15s, sys: 3.96 s, total: 1min 19s
Wall time: 1min 20s


**DECADE:**

Calculate zeropoints region by region...

In [11]:
%%time 

if do_calc_fgcm_psf_zps: 
    
    fluxObsColName =    'FLUX_PSF' 
    fluxerrObsColName = 'FLUXERR_PSF' 
    aggFieldColName =   'FILENAME' 
    magType =           'psf'
    
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']


    for regionName in region_name_list:
    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on region %s""" % (regionName)
        print """# # # # # # # # # # # # # # #"""
        print
        
        for band in bandList:
                    
            input_file_template = """cat_%s.%s.?.%s.fgcm_%s.fits""" % (subdir, regionName, band, magType)
            input_file_template = os.path.join(rawdata_dir, 'ExpCatFITS', subdir, input_file_template)
            input_file_list = glob.glob(input_file_template)
            input_file_list = np.sort(input_file_list)

            if np.size(input_file_list) == 0:
                print "No files matching template %s" % (input_file_template)
    
            for inputFile in input_file_list:
            
                print inputFile 

                if os.path.exists(inputFile):

                    #outputFile = os.path.splitext(inputFile)[0] + '.zps.csv'
                    #print outputFile
                    
                    outputFile = os.path.splitext(os.path.basename(inputFile))[0]+'.csv'
                    outputFile = 'zps_' + outputFile[4:]
                    outputFile = os.path.join(zeropoints_dir, outputFile)

                    print outputFile    
                    
                    status = DECam_tie_to_fgcm_stds(inputFile, outputFile, 
                                                    band, magType, 
                                                    fluxObsColName, fluxerrObsColName, 
                                                    aggFieldColName, verbose)
                    if status > 0:
                        print """ERROR:  %s FAILED!  Continuing...""" % (cmd)
            
                else:
            
                    print """%s does not exist...  skipping...""" % (inputFile)
                
                print


# # # # # # # # # # # # # # #
Working on region VVDSF14
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.g.fgcm_psf.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.r.fgcm_psf.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.i.fgcm_psf.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.z.fgcm_psf.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.Y.fgcm_psf.fits

# # # # # # # # # # # # # # #
Working on region VVDSF22
# # # # # # # # # # # # # # #

../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF22.0.g.fgcm_psf.fits
../Zeropoints/zps_DECADE_FINALCUT.VVDSF22.0.g.fgcm_psf.csv
2021-12-30 21:16:49.798887
Reading in ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF22.0.g.fgcm_psf.fits as a

2021-12-30 21:23:12.039840
2021-12-30 21:23:19.136554
2021-12-30 21:23:19.139408
  Number of rows remaining:  643861
   iter3...
2021-12-30 21:23:20.306532
2021-12-30 21:23:27.356851
2021-12-30 21:23:27.360686
  Number of rows remaining:  626315
2021-12-30 21:23:27.363220
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-30 21:23:27.580426

2021-12-30 21:23:31.046911
Writing ../Zeropoints/zps_DECADE_FINALCUT.SN-E.0.z.fgcm_psf.csv output file (using pandas to_csv method)...
2021-12-30 21:23:33.094974


No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-E.?.Y.fgcm_psf.fits

# # # # # # # # # # # # # # #
Working on region SN-X_err
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-X_err.?.g.fgcm_psf.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-X_err.?.r.fgcm_psf.fits
No files matching template ../RawDa

2021-12-30 21:26:08.315276


../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-S.0.i.fgcm_psf.fits
../Zeropoints/zps_DECADE_FINALCUT.SN-S.0.i.fgcm_psf.csv
2021-12-30 21:26:08.339448
Reading in ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-S.0.i.fgcm_psf.fits as a pandas DataFrame...
2021-12-30 21:26:16.669234

Sigma-clipping...
   iter1...
2021-12-30 21:26:17.505413
2021-12-30 21:26:20.205029
2021-12-30 21:26:20.206389
  Number of rows remaining:  225458
   iter2...
2021-12-30 21:26:20.710851
2021-12-30 21:26:23.338155
2021-12-30 21:26:23.339440
  Number of rows remaining:  219404
   iter3...
2021-12-30 21:26:23.733277
2021-12-30 21:26:26.395062
2021-12-30 21:26:26.398129
  Number of rows remaining:  213042
2021-12-30 21:26:26.401648
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-30 21:26:26.514937

2021-12-30 21:26:27.436200
Writing ../Zeropoints/zps_DECADE_FINALCUT.SN-S.0.i.fgcm_psf.csv output file (using pandas to_csv

2021-12-30 21:32:03.783600
2021-12-30 21:32:03.788352
  Number of rows remaining:  744968
   iter3...
2021-12-30 21:32:05.636240
2021-12-30 21:32:14.821984
2021-12-30 21:32:14.826903
  Number of rows remaining:  729971
2021-12-30 21:32:14.830328
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-30 21:32:15.087528

2021-12-30 21:32:19.073561
Writing ../Zeropoints/zps_DECADE_FINALCUT.EDFS.0.z.fgcm_psf.csv output file (using pandas to_csv method)...
2021-12-30 21:32:21.593542


No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.EDFS.?.Y.fgcm_psf.fits

# # # # # # # # # # # # # # #
Working on region MACS0416
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.MACS0416.?.g.fgcm_psf.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.MACS0416.?.r.fgcm_psf.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALC

Combine region-by-region results into a single file...

In [16]:
%%time 

if do_calc_fgcm_psf_zps: 
    
    magType = 'psf'
    
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']

    
    for band in bandList:
                    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
        print

        outputFile = """zps_%s.%s.fgcm_%s.csv""" % (subdir, band, magType)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
    
        input_file_template = """zps_%s.*.?.%s.fgcm_%s.csv""" % (subdir, band, magType)
        input_file_template = os.path.join(zeropoints_dir, input_file_template)
        input_file_list = glob.glob(input_file_template)
        input_file_list = np.sort(input_file_list)

        if np.size(input_file_list) == 0:
            print "No files matching template %s" % (input_file_template)
            continue
    
        df_comb = pd.concat(pd.read_csv(inputFile) for inputFile in input_file_list)

        outputFile = """zps_%s.%s.fgcm_%s.csv""" % (subdir, band, magType)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
        print outputFile
        df_comb.to_csv(outputFile, index=False)
        
        del df_comb



# # # # # # # # # # # # # # #
Working on band g
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.g.fgcm_psf.csv

# # # # # # # # # # # # # # #
Working on band r
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.r.fgcm_psf.csv

# # # # # # # # # # # # # # #
Working on band i
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.i.fgcm_psf.csv

# # # # # # # # # # # # # # #
Working on band z
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.z.fgcm_psf.csv

# # # # # # # # # # # # # # #
Working on band Y
# # # # # # # # # # # # # # #

No files matching template ../Zeropoints/zps_DECADE_FINALCUT.*.?.Y.fgcm_psf.csv
CPU times: user 15.9 s, sys: 696 ms, total: 16.6 s
Wall time: 16.6 s


## 5. Zeropoints by tying to FGCM Aper8 Standard Stars

**We will first work with the DES data, and then we will repeat for the DECADE data.**

**DES:**

Calculate zeropoints region by region...

In [18]:
%%time 

if do_calc_fgcm_aper8_zps: 
    
    fluxObsColName =    'FLUX_APER_08' 
    fluxerrObsColName = 'FLUXERR_APER_08' 
    aggFieldColName =   'FILENAME' 
    magType =           'aper8'
    
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']


    for regionName in region_name_list:
    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on region %s""" % (regionName)
        print """# # # # # # # # # # # # # # #"""
        print
        
        for band in bandList:
                    
            input_file_template = """cat_%s.%s.?.%s.fgcm_%s.fits""" % (subdir, regionName, band, magType)
            input_file_template = os.path.join(rawdata_dir, 'ExpCatFITS', subdir, input_file_template)
            input_file_list = glob.glob(input_file_template)
            input_file_list = np.sort(input_file_list)

            if np.size(input_file_list) == 0:
                print "No files matching template %s" % (input_file_template)
    
            for inputFile in input_file_list:
            
                print inputFile 

                if os.path.exists(inputFile):

                    #outputFile = os.path.splitext(inputFile)[0] + '.zps.csv'
                    #print outputFile
                    
                    outputFile = os.path.splitext(os.path.basename(inputFile))[0]+'.csv'
                    outputFile = 'zps_' + outputFile[4:]
                    outputFile = os.path.join(zeropoints_dir, outputFile)

                    print outputFile    
                    
                    status = DECam_tie_to_fgcm_stds(inputFile, outputFile, 
                                                    band, magType, 
                                                    fluxObsColName, fluxerrObsColName, 
                                                    aggFieldColName, verbose)
                    if status > 0:
                        print 'ERROR:  %s FAILED!  Continuing...'
            
                else:
            
                    print """%s does not exist...  skipping...""" % (inputFile)
                
                print


# # # # # # # # # # # # # # #
Working on region VVDSF14
# # # # # # # # # # # # # # #

../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.VVDSF14.0.g.fgcm_aper8.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.VVDSF14.0.g.fgcm_aper8.csv
2021-12-31 21:06:41.339571
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.VVDSF14.0.g.fgcm_aper8.fits as a pandas DataFrame...
2021-12-31 21:06:43.103768

Sigma-clipping...
   iter1...
2021-12-31 21:06:43.116552
2021-12-31 21:06:43.118629
2021-12-31 21:06:43.118787
  Number of rows remaining:  0
   iter2...
2021-12-31 21:06:43.122233
2021-12-31 21:06:43.123883
2021-12-31 21:06:43.123964
  Number of rows remaining:  0
   iter3...
2021-12-31 21:06:43.126707
2021-12-31 21:06:43.127991
2021-12-31 21:06:43.128124
  Number of rows remaining:  0
2021-12-31 21:06:43.128940
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-31 21:06:43.132269

2021-12-31 21:06:43.188310
Writing ../Zeropoints/zps_D

2021-12-31 21:07:14.772379

Sigma-clipping...
   iter1...
2021-12-31 21:07:18.311951
2021-12-31 21:07:26.774081
2021-12-31 21:07:26.778673
  Number of rows remaining:  837106
   iter2...
2021-12-31 21:07:29.764633
2021-12-31 21:07:38.191402
2021-12-31 21:07:38.195276
  Number of rows remaining:  816961
   iter3...
2021-12-31 21:07:39.953487
2021-12-31 21:07:48.247757
2021-12-31 21:07:48.251720
  Number of rows remaining:  798739
2021-12-31 21:07:48.254975
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-31 21:07:48.516201

2021-12-31 21:07:53.030040
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-E.0.g.fgcm_aper8.csv output file (using pandas to_csv method)...
2021-12-31 21:07:55.434137


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-E.0.r.fgcm_aper8.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-E.0.r.fgcm_aper8.csv
2021-12-31 21:07:55.642298
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-E.0.r.fgcm_aper

2021-12-31 21:21:26.363161
2021-12-31 21:21:46.671156
2021-12-31 21:21:46.679721
  Number of rows remaining:  1791115
   iter3...
2021-12-31 21:21:50.493747
2021-12-31 21:22:10.856038
2021-12-31 21:22:10.866539
  Number of rows remaining:  1747903
2021-12-31 21:22:10.872132
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-31 21:22:11.560705

2021-12-31 21:22:22.627809
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-X.0.i.fgcm_aper8.csv output file (using pandas to_csv method)...
2021-12-31 21:22:28.445579


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-X.1.i.fgcm_aper8.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-X.1.i.fgcm_aper8.csv
2021-12-31 21:22:29.185268
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-X.1.i.fgcm_aper8.fits as a pandas DataFrame...
2021-12-31 21:23:34.838105

Sigma-clipping...
   iter1...
2021-12-31 21:23:48.148906
2021-12-31 21:24:09.374303
2021-12-31 21:24:09.385628
  Number of ro

2021-12-31 21:39:11.799051

Sigma-clipping...
   iter1...
2021-12-31 21:39:12.313917
2021-12-31 21:39:13.474520
2021-12-31 21:39:13.475134
  Number of rows remaining:  104499
   iter2...
2021-12-31 21:39:13.649091
2021-12-31 21:39:14.788343
2021-12-31 21:39:14.788984
  Number of rows remaining:  101831
   iter3...
2021-12-31 21:39:14.982843
2021-12-31 21:39:16.130954
2021-12-31 21:39:16.131593
  Number of rows remaining:  99543
2021-12-31 21:39:16.132673
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-31 21:39:16.166061

2021-12-31 21:39:16.591154
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.ALHAMBRA2.0.i.fgcm_aper8.csv output file (using pandas to_csv method)...
2021-12-31 21:39:16.902209


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.ALHAMBRA2.0.z.fgcm_aper8.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.ALHAMBRA2.0.z.fgcm_aper8.csv
2021-12-31 21:39:16.927634
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.ALHA

2021-12-31 21:53:37.205259
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-C.0.r.fgcm_aper8.csv output file (using pandas to_csv method)...
2021-12-31 21:53:44.650363


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-C.0.i.fgcm_aper8.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.SN-C.0.i.fgcm_aper8.csv
2021-12-31 21:53:45.496022
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.SN-C.0.i.fgcm_aper8.fits as a pandas DataFrame...
2021-12-31 21:54:53.106718

Sigma-clipping...
   iter1...
2021-12-31 21:55:04.638914
2021-12-31 21:55:29.454693
2021-12-31 21:55:29.477167
  Number of rows remaining:  2184024
   iter2...
2021-12-31 21:55:38.982266
2021-12-31 21:56:03.483690
2021-12-31 21:56:03.495691
  Number of rows remaining:  2125870
   iter3...
2021-12-31 21:56:08.364858
2021-12-31 21:56:32.509820
2021-12-31 21:56:32.522639
  Number of rows remaining:  2066136
2021-12-31 21:56:32.530560
Performing grouping/aggregating functions on sigma-clipped pandas DataFr

2021-12-31 22:15:51.430813

Sigma-clipping...
   iter1...
2021-12-31 22:15:51.995752
2021-12-31 22:15:53.358781
2021-12-31 22:15:53.359528
  Number of rows remaining:  147755
   iter2...
2021-12-31 22:15:53.709005
2021-12-31 22:15:55.078013
2021-12-31 22:15:55.078736
  Number of rows remaining:  143851
   iter3...
2021-12-31 22:15:55.287447
2021-12-31 22:15:56.622396
2021-12-31 22:15:56.623085
  Number of rows remaining:  139941
2021-12-31 22:15:56.624183
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-31 22:15:56.670289

2021-12-31 22:15:57.252158
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.MACS0416.0.g.fgcm_aper8.csv output file (using pandas to_csv method)...
2021-12-31 22:15:57.681309


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.MACS0416.0.r.fgcm_aper8.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.MACS0416.0.r.fgcm_aper8.csv
2021-12-31 22:15:57.722396
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.MACS04

2021-12-31 22:17:55.076502
2021-12-31 22:17:59.067005
2021-12-31 22:17:59.068695
  Number of rows remaining:  408068
   iter3...
2021-12-31 22:17:59.586941
2021-12-31 22:18:03.511769
2021-12-31 22:18:03.513453
  Number of rows remaining:  398229
2021-12-31 22:18:03.515057
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2021-12-31 22:18:03.633467

2021-12-31 22:18:05.411169
Writing ../Zeropoints/zps_DES_Y6A2_FINALCUT.COSMOS.0.i.fgcm_aper8.csv output file (using pandas to_csv method)...
2021-12-31 22:18:06.487474


../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.COSMOS.0.z.fgcm_aper8.fits
../Zeropoints/zps_DES_Y6A2_FINALCUT.COSMOS.0.z.fgcm_aper8.csv
2021-12-31 22:18:06.544636
Reading in ../RawData/ExpCatFITS/DES_Y6A2_FINALCUT/cat_DES_Y6A2_FINALCUT.COSMOS.0.z.fgcm_aper8.fits as a pandas DataFrame...
2021-12-31 22:18:35.893614

Sigma-clipping...
   iter1...
2021-12-31 22:18:38.726220
2021-12-31 22:18:46.847685
2021-12-31 22:18:46.852991
  Number

Combine region-by-region results into a single file...

In [19]:
%%time 

if do_calc_fgcm_aper8_zps: 
    
    magType = 'aper8'
    
    subdir = """DES_%s""" % (tag_des)

    tmpdir = os.environ['TMPDIR']

    
    for band in bandList:
                    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
        print

        outputFile = """zps_%s.%s.fgcm_%s.csv""" % (subdir, band, magType)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
    
        input_file_template = """zps_%s.*.?.%s.fgcm_%s.csv""" % (subdir, band, magType)
        input_file_template = os.path.join(zeropoints_dir, input_file_template)
        input_file_list = glob.glob(input_file_template)
        input_file_list = np.sort(input_file_list)

        if np.size(input_file_list) == 0:
            print "No files matching template %s" % (input_file_template)
            continue
    
        df_comb = pd.concat(pd.read_csv(inputFile) for inputFile in input_file_list)

        outputFile = """zps_%s.%s.fgcm_%s.csv""" % (subdir, band, magType)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
        print outputFile
        df_comb.to_csv(outputFile, index=False)
        
        del df_comb



# # # # # # # # # # # # # # #
Working on band g
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.g.fgcm_aper8.csv

# # # # # # # # # # # # # # #
Working on band r
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.r.fgcm_aper8.csv

# # # # # # # # # # # # # # #
Working on band i
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.i.fgcm_aper8.csv

# # # # # # # # # # # # # # #
Working on band z
# # # # # # # # # # # # # # #

../Zeropoints/zps_DES_Y6A2_FINALCUT.z.fgcm_aper8.csv

# # # # # # # # # # # # # # #
Working on band Y
# # # # # # # # # # # # # # #

No files matching template ../Zeropoints/zps_DES_Y6A2_FINALCUT.*.?.Y.fgcm_aper8.csv
CPU times: user 1min 13s, sys: 2.52 s, total: 1min 16s
Wall time: 1min 16s


**DECADE:**

Calculate zeropoints region by region...

In [21]:
%%time 

if do_calc_fgcm_aper8_zps: 
    
    fluxObsColName =    'FLUX_APER_08' 
    fluxerrObsColName = 'FLUXERR_APER_08' 
    aggFieldColName =   'FILENAME' 
    magType =           'aper8'
    
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']


    for regionName in region_name_list:
    
        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on region %s""" % (regionName)
        print """# # # # # # # # # # # # # # #"""
        print
        
        for band in bandList:
                    
            input_file_template = """cat_%s.%s.?.%s.fgcm_%s.fits""" % (subdir, regionName, band, magType)
            input_file_template = os.path.join(rawdata_dir, 'ExpCatFITS', subdir, input_file_template)
            input_file_list = glob.glob(input_file_template)
            input_file_list = np.sort(input_file_list)

            if np.size(input_file_list) == 0:
                print "No files matching template %s" % (input_file_template)
    
            for inputFile in input_file_list:
            
                print inputFile 

                if os.path.exists(inputFile):

                    #outputFile = os.path.splitext(inputFile)[0] + '.zps.csv'
                    #print outputFile
                    
                    outputFile = os.path.splitext(os.path.basename(inputFile))[0]+'.csv'
                    outputFile = 'zps_' + outputFile[4:]
                    outputFile = os.path.join(zeropoints_dir, outputFile)

                    print outputFile    
                    
                    status = DECam_tie_to_fgcm_stds(inputFile, outputFile, 
                                                    band, magType, 
                                                    fluxObsColName, fluxerrObsColName, 
                                                    aggFieldColName, verbose)
                    if status > 0:
                        print """ERROR:  %s FAILED!  Continuing...""" % (cmd)
            
                else:
            
                    print """%s does not exist...  skipping...""" % (inputFile)
                
                print


# # # # # # # # # # # # # # #
Working on region VVDSF14
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.g.fgcm_aper8.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.r.fgcm_aper8.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.i.fgcm_aper8.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.z.fgcm_aper8.fits
No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF14.?.Y.fgcm_aper8.fits

# # # # # # # # # # # # # # #
Working on region VVDSF22
# # # # # # # # # # # # # # #

../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF22.0.g.fgcm_aper8.fits
../Zeropoints/zps_DECADE_FINALCUT.VVDSF22.0.g.fgcm_aper8.csv
2022-01-02 18:03:07.832225
Reading in ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.VVDSF22.0.g.fgcm

2022-01-02 18:08:57.845838
2022-01-02 18:09:05.101632
2022-01-02 18:09:05.106633
  Number of rows remaining:  661179
   iter2...
2022-01-02 18:09:06.847076
2022-01-02 18:09:13.946603
2022-01-02 18:09:13.949955
  Number of rows remaining:  643504
   iter3...
2022-01-02 18:09:15.124872
2022-01-02 18:09:22.194695
2022-01-02 18:09:22.198103
  Number of rows remaining:  625623
2022-01-02 18:09:22.200305
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-02 18:09:22.416163

2022-01-02 18:09:25.932961
Writing ../Zeropoints/zps_DECADE_FINALCUT.SN-E.0.z.fgcm_aper8.csv output file (using pandas to_csv method)...
2022-01-02 18:09:28.019655


No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-E.?.Y.fgcm_aper8.fits

# # # # # # # # # # # # # # #
Working on region SN-X_err
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-X_err.?.g.fgcm_aper8.fits
No files mat

2022-01-02 18:12:01.249582
Writing ../Zeropoints/zps_DECADE_FINALCUT.SN-S.0.r.fgcm_aper8.csv output file (using pandas to_csv method)...
2022-01-02 18:12:01.721077


../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-S.0.i.fgcm_aper8.fits
../Zeropoints/zps_DECADE_FINALCUT.SN-S.0.i.fgcm_aper8.csv
2022-01-02 18:12:01.755474
Reading in ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.SN-S.0.i.fgcm_aper8.fits as a pandas DataFrame...
2022-01-02 18:12:09.811831

Sigma-clipping...
   iter1...
2022-01-02 18:12:10.798172
2022-01-02 18:12:13.472755
2022-01-02 18:12:13.473967
  Number of rows remaining:  226295
   iter2...
2022-01-02 18:12:14.065706
2022-01-02 18:12:16.651144
2022-01-02 18:12:16.652409
  Number of rows remaining:  220358
   iter3...
2022-01-02 18:12:17.010320
2022-01-02 18:12:19.658207
2022-01-02 18:12:19.659452
  Number of rows remaining:  214308
2022-01-02 18:12:19.661090
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-

2022-01-02 18:17:40.085614

Sigma-clipping...
   iter1...
2022-01-02 18:17:42.697385
2022-01-02 18:17:51.776592
2022-01-02 18:17:51.781403
  Number of rows remaining:  762462
   iter2...
2022-01-02 18:17:54.054045
2022-01-02 18:18:02.963866
2022-01-02 18:18:02.969042
  Number of rows remaining:  742324
   iter3...
2022-01-02 18:18:04.486196
2022-01-02 18:18:13.425876
2022-01-02 18:18:13.430165
  Number of rows remaining:  722970
2022-01-02 18:18:13.434902
Performing grouping/aggregating functions on sigma-clipped pandas DataFrame...
2022-01-02 18:18:13.680960

2022-01-02 18:18:17.975402
Writing ../Zeropoints/zps_DECADE_FINALCUT.EDFS.0.z.fgcm_aper8.csv output file (using pandas to_csv method)...
2022-01-02 18:18:20.558930


No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat_DECADE_FINALCUT.EDFS.?.Y.fgcm_aper8.fits

# # # # # # # # # # # # # # #
Working on region MACS0416
# # # # # # # # # # # # # # #

No files matching template ../RawData/ExpCatFITS/DECADE_FINALCUT/cat

Combine region-by-region results into a single file...

In [22]:
%%time 

if do_calc_fgcm_aper8_zps: 
    
    magType = 'aper8'
    
    subdir = """%s""" % (tag_decade)

    tmpdir = os.environ['TMPDIR']

    
    for band in bandList:

        print
        print """# # # # # # # # # # # # # # #"""
        print """Working on band %s""" % (band)
        print """# # # # # # # # # # # # # # #"""
        print

        outputFile = """zps_%s.%s.fgcm_%s.csv""" % (subdir, band, magType)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
    
        input_file_template = """zps_%s.*.?.%s.fgcm_%s.csv""" % (subdir, band, magType)
        input_file_template = os.path.join(zeropoints_dir, input_file_template)
        input_file_list = glob.glob(input_file_template)
        input_file_list = np.sort(input_file_list)

        if np.size(input_file_list) == 0:
            print "No files matching template %s" % (input_file_template)
            continue
    
        df_comb = pd.concat(pd.read_csv(inputFile) for inputFile in input_file_list)

        outputFile = """zps_%s.%s.fgcm_%s.csv""" % (subdir, band, magType)
        outputFile = outputFile = os.path.join(zeropoints_dir, outputFile)
        print outputFile
        df_comb.to_csv(outputFile, index=False)
        
        del df_comb



# # # # # # # # # # # # # # #
Working on band g
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.g.fgcm_aper8.csv

# # # # # # # # # # # # # # #
Working on band r
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.r.fgcm_aper8.csv

# # # # # # # # # # # # # # #
Working on band i
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.i.fgcm_aper8.csv

# # # # # # # # # # # # # # #
Working on band z
# # # # # # # # # # # # # # #

../Zeropoints/zps_DECADE_FINALCUT.z.fgcm_aper8.csv

# # # # # # # # # # # # # # #
Working on band Y
# # # # # # # # # # # # # # #

No files matching template ../Zeropoints/zps_DECADE_FINALCUT.*.?.Y.fgcm_aper8.csv
CPU times: user 16.3 s, sys: 719 ms, total: 17.1 s
Wall time: 17.2 s
